In [2]:
!pip install langchain langchain-community langchain-core requests langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.43.0
    Uninstalling google-auth-2.43.0:
 

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [2]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [3]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url =f'https://v6.exchangerate-api.com/v6/671717fca61a5a67920b1e08/latest/{base_currency}'

  response = requests.get(url).json()
  conversion_factor=response["conversion_rates"][target_currency]

  return conversion_factor

@tool
def convert(base_currency_value: int, conversion_factor: Annotated[float,InjectedToolArg])-> float:
  """This calculates the amount of base currency into equivalent value in target currency using conversion rate"""
  return base_currency_value * conversion_factor


In [4]:
val=get_conversion_factor.invoke({"base_currency":"USD","target_currency":"AED"})
print(val)

3.6725


In [5]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"]=userdata.get("GOOGLE_API_KEY")

In [6]:
llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")
llm_with_tools=llm.bind_tools([convert,get_conversion_factor])


In [7]:
# query=HumanMessage("convert 5 USD into AED")
# message=[]
# message.append(query)

messages = [HumanMessage('What is the conversion factor between USD and AED, and based on that can you convert 10 USD to AED')]

In [8]:
ai_message=llm_with_tools.invoke(messages)

In [53]:
ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'conversion', 'arguments': '{"base_currency_value": 10}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b68f5-2b14-7560-8379-aa87c1e27144-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'AED'}, 'id': '152537a0-3cf9-4940-a28b-b38cb8ea3ab9', 'type': 'tool_call'}, {'name': 'conversion', 'args': {'base_currency_value': 10}, 'id': '395b46a7-4b7d-4908-aa28-17aafe060224', 'type': 'tool_call'}], usage_metadata={'input_tokens': 147, 'output_tokens': 44, 'total_tokens': 191, 'input_token_details': {'cache_read': 0}})

In [9]:
messages.append(ai_message)

In [10]:
messages

[HumanMessage(content='What is the conversion factor between USD and AED, and based on that can you convert 10 USD to AED', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert', 'arguments': '{"base_currency_value": 10}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b692d-06a4-7270-9bc1-357ac4e3f96d-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'AED'}, 'id': '217341da-bb93-4e8e-8e61-dd977a99eef7', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10}, 'id': '387658de-f2ae-4565-a94f-05aec537ecb6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 147, 'output_tokens': 44, 'total_tokens': 191, 'input_token_details': {'cache_read': 0}})]

In [11]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'AED'},
  'id': '217341da-bb93-4e8e-8e61-dd977a99eef7',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': '387658de-f2ae-4565-a94f-05aec537ecb6',
  'type': 'tool_call'}]

In [12]:
tool_call1=get_conversion_factor.invoke(messages[1].tool_calls[0])

In [13]:
tool_call1

ToolMessage(content='3.6725', name='get_conversion_factor', tool_call_id='bd372271-6d3f-4316-b493-fcdca7a66af2')

In [12]:
messages.append(tool_call1)

In [38]:
llm_with_tools.invoke(messages)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'conversion', 'arguments': '{"base_currency_value": 10, "conversation_factor": 3.6725}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b68ef-5192-7001-8acb-7b301df17a7e-0', tool_calls=[{'name': 'conversion', 'args': {'base_currency_value': 10, 'conversation_factor': 3.6725}, 'id': '33b8534c-788a-4fea-8e51-2f0194871fb1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 238, 'output_tokens': 30, 'total_tokens': 268, 'input_token_details': {'cache_read': 0}})

In [14]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'AED', 'base_currency': 'USD'},
  'id': 'bd372271-6d3f-4316-b493-fcdca7a66af2',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'f5ad4053-25fc-46b1-97a0-d7ad364bee94',
  'type': 'tool_call'}]

In [15]:
get_conversion_factor.invoke(ai_message.tool_calls[0])

ToolMessage(content='3.6725', name='get_conversion_factor', tool_call_id='bd372271-6d3f-4316-b493-fcdca7a66af2')

In [21]:
tool_call=ai_message.tool_calls[0]
tool_message1 = get_conversion_factor.invoke(tool_call)
print(tool_message1)

content='3.6725' name='get_conversion_factor' tool_call_id='bd372271-6d3f-4316-b493-fcdca7a66af2'


In [12]:
messages

[HumanMessage(content='What is the conversion factor between USD and AED, and based on that can you convert 10 USD to AED', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert', 'arguments': '{"base_currency_value": 10}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b692d-06a4-7270-9bc1-357ac4e3f96d-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'AED'}, 'id': '217341da-bb93-4e8e-8e61-dd977a99eef7', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10}, 'id': '387658de-f2ae-4565-a94f-05aec537ecb6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 147, 'output_tokens': 44, 'total_tokens': 191, 'input_token_details': {'cache_read': 0}})]

In [15]:
# 1. First, find and execute the conversion factor tool to get the rate
conversion_rate = None

for tool_call in ai_message.tool_calls:
    if tool_call['name'] == 'get_conversion_factor':
        tool_message = get_conversion_factor.invoke(tool_call)
        messages.append(tool_message)
        conversion_rate = tool_message.content

# 2. Now that we have the rate, execute the 'convert' tool
for tool_call in ai_message.tool_calls:
    if tool_call['name'] == 'convert':
        if conversion_rate is not None:
            # Inject the dependency manually into the arguments
            tool_call['args']['conversion_factor'] = float(conversion_rate)

            tool_message = convert.invoke(tool_call)
            messages.append(tool_message)
        else:
            print("Error: Could not perform conversion because rate was not fetched.")

In [16]:
messages


[HumanMessage(content='What is the conversion factor between USD and AED, and based on that can you convert 10 USD to AED', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert', 'arguments': '{"base_currency_value": 10}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b692d-06a4-7270-9bc1-357ac4e3f96d-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'AED'}, 'id': '217341da-bb93-4e8e-8e61-dd977a99eef7', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10, 'conversion_factor': 3.6725}, 'id': '387658de-f2ae-4565-a94f-05aec537ecb6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 147, 'output_tokens': 44, 'total_tokens': 191, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='3.6725', name='get_conversion_factor', 

In [17]:
llm_with_tools.invoke(messages)

AIMessage(content='The conversion factor between USD and AED is 3.6725. 10 USD is equivalent to 36.725 AED.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b6936-ec2a-7cf0-bb1f-83f3f84cc5b4-0', usage_metadata={'input_tokens': 295, 'output_tokens': 32, 'total_tokens': 327, 'input_token_details': {'cache_read': 0}})

In [19]:
print(llm_with_tools.invoke(messages).content)

The conversion factor between USD and AED is 3.6725. 10 USD is equal to 36.725 AED.
